In [1]:
from collections import OrderedDict
from pymatgen import MPRester
import requests
import json

/home/azadoks/miniconda3/envs/python2/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


In [2]:
material_ids = OrderedDict([
    ('diamond', 'mp-66'),  # C
    ('gaas', 'mp-2534'),  # GaAs
    ('rocksalt', 'mp-22862'),  # NaCl
    ('cubic_perovskite', 'mp-2998'),  # BaTiO3
    ('tetragonal_perovskite', 'mp-5986'),  # BaTiO3
    ('trigonal_perovskite', 'mp-5020'),  # BaTiO3
    ('orthorhombic_perovskite', 'mp-5777'),  # BaTiO3
    ('zincblende', 'mp-10695'),  # ZnS
    ('wurtzite', 'mp-10281'),  # ZnS
    ('fcc', 'mp-23'),  # Ni
    ('big_fcc', 'mp-76'),  # Sr
    ('bcc', 'mp-13'),  # Fe
    ('big_bcc', 'mp-70'),  # Rb
    ('hcp', 'mp-153'),  # Mg
    ('trigonal', 'mp-782'),  # Te2Pd
    ('tetragonal', 'mp-742'),  # Ti2Cu
    ('monoclinic', 'mp-684'),  # BaS2
    ('triclinic', 'mp-9122'),  # CaP3
    ('orthorhombic', 'mp-872')  # BaSn
])

In [3]:
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures = OrderedDict([(name, mpr.get_structure_by_material_id(id)) for name, id in material_ids.iteritems()])

In [ ]:
payloads = {
    'soaps': payload = {
        'structures': json.dumps([structures[s].as_dict() for s in structures.values()]),
        'nocenters': None, 'chem_channels': False,
        'centerweight': 1.0, 'gaussian_width': 0.5,
        'cutoff': 3.5, 'cutoff_transition_width': 0.5,
        'nmax': 8, 'lmax': 6,
        'spkitMax': None, 'chemicalProjection': None,
        'is_fast_average': False
    },
    'matminer': {
        'structures': json.dumps([s.as_dict() for s in structures.values()]),
        'preset': 'cn',
        'crystal_site_args': {},
        'site_stats_args': {}
    },
    'pymatgen': {
        'structures': json.dumps([s.as_dict() for s in structures.values()]),
        'comparator': 'OccupancyComparator',
        'anonymous': True,
    },
    'stidy': {
        'structures': json.dumps([s.as_dict() for s in structures.values()]),
        'symprec': 1e-2,
        'angle_tolerance': 5.
    }
}

In [ ]:
endpoint = "fingerprints"
function = "soaps"
payload = {
    'atoms': json.dumps([s.as_dict() for s in structures.values()]),
    'nocenters': None, 'chem_channels': False,
    'centerweight': 1.0, 'gaussian_width': 0.5,
    'cutoff': 3.5, 'cutoff_transition_width': 0.5,
    'nmax': 8, 'lmax': 6,
    'spkitMax': None, 'chemicalProjection': None,
    'is_fast_average': False
}

r = requests.get('http://127.0.0.1:5000/v1/{}/{}'.format(endpoint, function), params=payload)

In [11]:
endpoint = "fingerprints"
function = "matminer"
payload = {
    'structures': json.dumps([s.as_dict() for s in structures.values()]),
    'preset': 'cn',
    'crystal_site_args': {},
    'site_stats_args': {}
}

r = requests.get('http://127.0.0.1:5000/v1/{}/{}'.format(endpoint, function), params=payload)
print(r)

<Response [404]>


In [ ]:
endpoints = {# 'distances': ['matminer', 'pymatgen', 'soaps'],
             # 'comparisons': ['matminer', 'pymatgen', 'soaps'],
             # 'fingerprints': ['matminer', 'stidy', 'soaps'],
             'fingerprints': ['soaps']}
for endpoint, functions in endpoints.items():
    for function in functions:
        print(endpoint, function)
        payload = payloads[function]
        r = requests.get('http://127.0.0.1:5000/v1/{}/{}'.format(endpoint, function), params=payload)
        results = json.loads(r.text)
        print(results)